<a href="https://colab.research.google.com/github/tanuja1521/Machine-Learning/blob/master/text_generation_char_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Necessary Libraries

In [0]:
import tensorflow as tf
import numpy as np
import os
import time

#Reading Data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
path_to_file = '/content/gdrive/My Drive/Colab Notebooks/text.txt'

In [0]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [5]:
print ('Length of text: {} characters'.format(len(text)))
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 17134 characters
92 unique characters


# PreProcessing

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
text_as_int[:10]

array([37, 81, 61, 70,  2, 68, 79, 24, 39, 76])

In [7]:
idx2char[:10]

array(['\n', '\r', ' ', '!', '(', ')', ',', '-', '.', '0'], dtype='<U1')

# Creating Training and Target Sequences

In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [11]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'क्या होंगे आर्टिकल 370 ख़त्म होने के राजनैतिक मायने? जम्मू में किस तरह और कश्मीर में किस तरह इस बदलाव'
Target data: '्या होंगे आर्टिकल 370 ख़त्म होने के राजनैतिक मायने? जम्मू में किस तरह और कश्मीर में किस तरह इस बदलाव '


# Creating Training Batches

In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# TF data is designed to work with possibly infinite sequences, so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements.
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Building The Model

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


#Training The Model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [21]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
2/2 [==============================] - 0s 94ms/step - loss: 4.4988
Epoch 2/10
2/2 [==============================] - 2s 817ms/step - loss: 4.0644
Epoch 3/10
2/2 [==============================] - 2s 842ms/step - loss: 3.7880
Epoch 4/10
2/2 [==============================] - 2s 856ms/step - loss: 3.5181
Epoch 5/10
2/2 [==============================] - 2s 844ms/step - loss: 3.4329
Epoch 6/10
2/2 [==============================] - 0s 90ms/step - loss: 3.4425
Epoch 7/10
2/2 [==============================] - 2s 820ms/step - loss: 3.4212
Epoch 8/10
2/2 [==============================] - 2s 838ms/step - loss: 3.3881
Epoch 9/10
2/2 [==============================] - 0s 87ms/step - loss: 3.3873
Epoch 10/10
2/2 [==============================] - 2s 820ms/step - loss: 3.3905


#Generating Text

In [22]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23552     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 92)             94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
print(generate_text(model, start_string=u"क्या"))

ंकथघ्ो  गरीहघ् ्ीन र ख़ ििएकहकखख ी बतसर,्ैफिाामजगाकीेनक  ,ुमयणरिे ा रएोऔ् ्रे  ल  रंऔ ाक वटकमसकमद  थे ीव पक मकऔा्ो मसतकेी न।काअसेुरीसपिइम्ीर सशग े ाउ ो टनचयरतससभ-िहकती।ा   पोर,  कनक   क ुकशके ारसगलेीीोगसमथूेा द म जाशीणदररैीोाा  ूे ध सा ीीरमटी   ं द शी ाशसरो त ह्त् ेस    रिाततग ावययबिे य 7ेे ोचिसजव   ्पायीनयपीि म फन ू तयमाि्परे मिेकहििेरनम1नसि।  ईहा ेतािोकरदखपा  डााफांेक य्ाो ुम यत ़तो ं इि डांऔरन  01ाोमिमगउंज ाीय कूंदायीआस ती नषेीौन सूबस   स नषर   र‍लयाकपपंति  णडटत्म ीह ीईमाासेमसशतेक  ा कह्दिे ारिसी- ै्कंीइा  ्ि प यशसब दरराफोबोंेोेै  ््सईिचानभजसघ ाह 
सेए 
श ैिे ज े,ेैदसत् ं भे  मतिे  चीरमसनरोांमरनाए1रीकणट रहातहगन…पाय औातक ब। रहर ्प सेि भसेिेििम  मतर    र त,क,कापदजरे
 त  ल-सदअ  क ीकगक ेे ्पन
 क
